In [ ]:
import igraph 
import csv
import unicodedata
import numpy as np

In [2]:
def readCSVToIterableCollection(path):
    print("Reading file " + path + ", begin...")
    collection = []
    with open(path, mode = 'r', encoding = "utf8") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter = ';')
        line_count = 0
        for row in csv_reader:
            if line_count > 0:
                collection.append(row)
            line_count += 1
    
    print("Data lines read: " + str(line_count - 1))
    return collection

In [ ]:
def readVertices_NBA(collection, graph, addNames):
    print("Reading NBA vertices, begin...")
    for row in collection:
        string = unicodedata.normalize('NFKD', row[1]).replace(u'ł', 'l').encode('ascii', 'ignore')
        if addNames:
            graph.add_vertex(name = string.decode('utf-8'))
        else:
            graph.add_vertex()
            
    print("Vertices read: " + str(len(g.vs)))

In [1]:
def readEdges_NBA(collection, graph, addYearsAsWeight):
    print("Reading NBA edges, begin...")
    edges = []
    weights = []
    for row in collection:
        year = row[2][0:4]
        edges.append((int(row[1])-1, int(row[0])-1))
        if addYearsAsWeight:
            weights.append(2020-int(year))
            
    graph.add_edges(edges)
    graph.es["weight"] = weights
    print("Edges read: " + str(len(graph.es)))

In [ ]:
def getSeasons_NBA(collection):
    return np.array(collection)[:,2].tolist()

In [ ]:
def getFirstClub_NBA(path, graph):
    return np.array(collection)[:,3].tolist()

In [ ]:
def readVertices_Filmweb(collection, graph):
    print("Reading Filmweb vertices, begin...")
    for row in collection:
        graph.add_vertex(name = row[2])
        
    print("Vertices read: " + str(len(graph.vs)))


In [1]:
def readEdges_Filmweb(collection, graph, addCommonMoviesAsWeight, addYearsKnownAsWeight):
    for row in collection:
        label1 = graph.vs["name"].index(row[0])
        label2 = graph.vs["name"].index(row[1])
        if addCommonMoviesAsWeight:
            edge=graph.add_edge(label1,label2)
            edge["weight"]=int(row[3])
        elif addYearsKnownAsWeight:
            edge=graph.add_edge(label1,label2)
            edge["weight"]=2020-int(row[2])
        else:
            graph.add_edges([(label1,label2)])

In [ ]:
def getNames_Filmweb(collection):
    names=[]
    for row in collection:
        string = row[0]
        string = unicodedata.normalize('NFKD', string).replace(u'ł', 'l').encode('ascii', 'ignore')
        names.append(string.decode('utf-8'))
    return names
    

In [ ]:
def getCommonMoviesCount_Filmweb(collection):
    return np.array(collection)[:,3].tolist()

In [ ]:
def getDates_Filmweb(collection):
    return np.array(collection)[:,2].tolist()